### `--Did not manage to complete-- Testing MLFlow and Streamlit`
---

# Initialize MLFlow
1. Open a new terminal window
1. Activate the `dsi-sg` mamba/conda environment
1. Navigate to the `12.01-mlops/solution-code` lesson folder
1. Now run the below code from this terminal window. Ensure you're inside the `solution-code` folder!: `mlflow ui --backend-store-uri sqlite:///mlflow.db`
1. The above code starts a small webserver running the MLFlow application on your computer. It uses SQLite to store the experiment information in a database
1. You can open the mlflow UI by opening this link in your browser: http://localhost:5000/

<img src="../images/01_01_MLFlow_UI.png" width="1000">

In [ ]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")

# Set the name of the experiment we're running in this notebook
# MLFlow will connect to an existing experiment or create a new one if the experiment is not already present
mlflow.set_experiment("dunkin_or_starbucks")

# Start automatically logging experiment details to MLFlow
mlflow.autolog()

2022/10/01 08:55:46 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2022/10/01 08:55:47 INFO mlflow.tracking.fluent: Autologging successfully enabled for statsmodels.


In [ ]:
X = df_concat['stem'] # still on the stemmed column
y = df_concat['subreddit']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.3, 
    random_state=42, 
    stratify=y
    )

In [ ]:
#instanciating the object
rf = RandomForestClassifier() 

In [ ]:
tvec = TfidfVectorizer()#instanciating the object

In [ ]:
X_train = tvec.fit_transform(X_train)

In [ ]:
X_train.shape

(4060, 10470)

In [ ]:
X_test =  tvec.fit(X_test) #no fit_transform here, only fit

In [ ]:
#Preping for the gridsearch with random forest params
rf_params = {
    'n_estimators': [100, 150], # iterating over different number of trees in the forest
    'max_depth': [None, 1, 2]# iterating over a defined set of max depth of tree. For None, the nodes are expanded until all leaves are pure
}

In [ ]:
%%time
#using GridsearchCv to find the best parameters for this model
gscv_rf = GridSearchCV(
    rf, 
    param_grid=rf_params, 
    cv=3, #cross validate our data
    verbose =1)

CPU times: total: 0 ns
Wall time: 0 ns


In [ ]:
%%time
gscv_rf.fit(X_train, y_train)

2022/09/20 20:29:10 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'f63fe607849645379e4881df0f6e9abd', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


Fitting 3 folds for each of 6 candidates, totalling 18 fits


2022/09/20 20:30:33 INFO mlflow.sklearn.utils: Logging the 5 best runs, one run will be omitted.


CPU times: total: 57 s
Wall time: 1min 23s


GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [None, 1, 2], 'n_estimators': [100, 150]},
             verbose=1)

In [ ]:
print(gscv_rf.best_score_)
print(gscv_rf.best_params_)

0.8467963127328332
{'max_depth': None, 'n_estimators': 150}


In [ ]:
from sklearn.metrics import get_scorer

In [ ]:
with mlflow.start_run():
    knn_model.fit(X_train, y_train)

In [ ]:
from mlflow.artifacts import download_artifacts

In [ ]:
# Download the desired model from MLFlow to local directory
# Get the URL by following instructions in above image
full_path = './mlruns/2/208134a1a927409b99259d328f403aad/artifacts/model'

In [ ]:
download_artifacts(full_path, dst_path='.')

'c:\\Users\\galvi\\OneDrive\\Documents\\GeneralAssembly\\Projects\\project_3\\model'

## `On to Flask`
---

In [ ]:
%%writefile inference.py
from flask import Flask, request
import pandas as pd
import os
import mlflow.pyfunc

# Step 2: Instantiate the Flask API
api = Flask('ModelEndpoint')

# Step 3: Load the model
model = mlflow.pyfunc.load_model(model_uri="./model")

# Step 4: Create the routes
## route 1: Health check. Just return success if the API is running
@api.route('/')
def home():
    # return a simple string
    return {"message": "Hi there!", "success": True}, 200

    # route 2: accept input data
# Post method is used when we want to receive some data from the user
@api.route('/predict', methods = ['POST'])
def make_predictions():
    # Get the data sent over the API
    user_input = request.get_json(force=True)
    
    # Convert user inputs to pandas dataframe
    df_schema = {"selftext":str} # To ensure the columns get the correct datatype
    user_input_df = pd.read_json(user_input, lines=True, dtype=df_schema) # Convert JSONL to dataframe
    
    # Run predictions and convert to list
    predictions = model.predict(user_input_df).tolist()
    
    return {'predictions': predictions}
    

# Step 5: Main function that actually runs the API!
if __name__ == '__main__':
    api.run(host='0.0.0.0', 
            debug=True, # Debug=True ensures any changes to inference.py automatically updates the running API
            port=int(os.environ.get("PORT", 8080))
           ) 

Overwriting inference.py


## `On to docker`
------

In [ ]:
%%writefile Dockerfile
# Use the official lightweight Python image from
# https://hub.docker.com/_/python
FROM python:3.8-slim

# Copy all the files needed for the app to work
COPY inference.py .
COPY model/ ./model

# Install all the necessary libraries
RUN pip install -r ./model/requirements_test.txt

# Run the API!
CMD python inference.py

Overwriting Dockerfile


In [ ]:
%%writefile model/requirements_test.txt
pandas
flask
mlflow-skinny
scikit-learn==1.1.1

Overwriting model/requirements_test.txt


In [ ]:
#in git bash
#gcloud init
#gcloud run deploy dunkin_or_starbucks --source . --region asia-southeast1

## `On to Streamlit`
-----


In [ ]:
%%writefile streamlit_app.py
import streamlit as st
import requests
import json

# Title of the page
st.title("🎓 DunkinOrStarbucks")

Overwriting streamlit_app.py


In [ ]:
%%writefile -a streamlit_app.py

# Get user inputs
comment  =st.text("Comment", help ="paste your comment here to check whether it belongs to Dunkin or Starbucks subreddit" )


# Display the inputs
user_input = {"comment":comment}
# st.write("User input:")
st.write(user_input)

Appending to streamlit_app.py


In [ ]:
%%writefile -a streamlit_app.py

# Code to post the user inputs to the API and get the predictions
# Paste the URL to your GCP Cloud Run API here!
api_url = 'https://dunkinorstarbucks-jqjv2zme2q-as.a.run.app'
api_route = '/predict'

response = requests.post(f'{api_url}{api_route}', json=json.dumps(user_input)) # json.dumps() converts dict to JSON
predictions = response.json()

# Add a submit button
if st.button("Submit"): 
    st.write(f"Prediction: {predictions['predictions'][0]}")

Appending to streamlit_app.py


## `Next steps`
---
1. Shipping the product to **streamlist** (input a comment and check whether it belongs to dunkin or starbuck)
2. Enhancing the model with **feature engineering**, trying out new models like **SMV, bagging, log regression, single decision
trees
3. fetch more data